# Chapter 4 Exercise 1: 表示学习基础 - 为深度因果模型铺路

## 学习目标

1. 理解为什么需要学习表示 (Representation Learning)
2. 掌握简单的神经网络特征提取
3. 理解处理组和对照组表示的差异
4. 为深度因果模型打下基础

---

## 从传统机器学习到深度因果模型

### 传统方法的局限

我们之前学习的 S-Learner、T-Learner 等方法都依赖于 **手工特征**。但现实世界中：

- 特征可能是非线性的
- 真正有用的信息可能隐藏在特征的复杂组合中
- 高维数据 (如图像、文本) 无法直接使用

### 表示学习的魔力

**表示学习 (Representation Learning)** 让神经网络自动学习最有用的特征表示：

$$X \xrightarrow{\text{神经网络}} \Phi(X) \xrightarrow{\text{因果模型}} Y(0), Y(1)$$

这个 $\Phi(X)$ 就是学习到的 "表示"！

## 生活化类比：翻译器

想象你是一个只懂中文的人，要判断一篇英文论文的质量。

**传统方法**：
- 手工提取特征：单词数、句子长度、标点符号数...
- 这些特征可能和论文质量关系不大

**表示学习方法**：
- 先用神经网络把英文 "翻译" 成一种通用表示
- 这个表示捕获了论文的语义信息
- 然后用这个表示来预测论文质量

神经网络就像一个 **智能翻译器**，把原始数据翻译成对任务最有用的形式！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from typing import Tuple
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

print(f"PyTorch 版本: {torch.__version__}")
print("环境准备完成！")

---

## Part 1: 非线性数据的挑战

### 为什么线性模型不够用?

假设真实的数据生成过程是非线性的：

- 原始特征: $X_1, X_2 \sim N(0, 1)$
- 真正有用的特征: $\Phi_1 = \sin(X_1)$, $\Phi_2 = X_1 \times X_2$
- 处理分配: $P(T=1|X) = \sigma(\Phi_1 + \Phi_2)$
- 结果: $Y = 1 + 2T + \Phi_1 + 0.5\Phi_2 + \epsilon$

如果我们只用 $X_1, X_2$ 建模，会错过关键的非线性关系！

In [ ]:
# 练习 1.1: 生成非线性数据

def generate_nonlinear_data(
    n: int = 1000,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    生成非线性数据: 原始特征无法直接捕获因果关系
    
    DGP (Data Generating Process):
    - X1, X2 ~ N(0, 1)
    - 真实有用特征: Phi1 = sin(X1), Phi2 = X1 * X2
    - T ~ Bernoulli(sigmoid(Phi1 + Phi2))
    - Y = 1 + 2*T + Phi1 + 0.5*Phi2 + noise
    """
    np.random.seed(seed)
    
    # 生成原始特征 X1, X2
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    
    # 计算有用特征
    Phi1 = np.sin(X1)
    Phi2 = X1 * X2
    
    # 生成处理 T (通过 sigmoid 函数)
    logit = Phi1 + 0.5 * Phi2
    propensity = 1 / (1 + np.exp(-logit))
    T = np.random.binomial(1, propensity, n)
    
    # 生成结果 Y
    noise = np.random.randn(n) * 0.5
    Y = 1 + 2*T + Phi1 + 0.5*Phi2 + noise
    
    X = np.column_stack([X1, X2])
    
    return X, T, Y

# 测试
X, T, Y = generate_nonlinear_data()
print(f"数据形状: X={X.shape}, T={T.shape}, Y={Y.shape}")
print(f"处理比例: {T.mean():.2%}")
print(f"平均结果: {Y.mean():.4f}")
print(f"\n真实 ATE = 2.0 (这是我们设定的)")

In [ ]:
# 练习 1.1 续: 朴素线性估计

def naive_linear_estimation(X, T, Y):
    """
    使用线性回归估计 ATE (直接使用原始特征)
    
    提示: Y = beta0 + beta1*T + beta2*X1 + beta3*X2 + epsilon
          beta1 就是 ATE 估计
    """
    # 构造特征矩阵 [T, X]
    features = np.column_stack([T, X])
    
    # 训练线性回归
    model = LinearRegression()
    model.fit(features, Y)
    
    # 返回 T 的系数作为 ATE 估计
    ate_estimate = model.coef_[0]  # T 是第一个特征
    
    return ate_estimate

# 测试
ate_linear = naive_linear_estimation(X, T, Y)
print(f"线性回归估计的 ATE: {ate_linear:.4f}")
print(f"真实 ATE: 2.0")
print(f"误差: {abs(ate_linear - 2.0):.4f}")
print(f"\n由于忽略了非线性关系，线性估计可能有偏差!")

---

## Part 2: 简单的表示学习网络

### 神经网络架构

```
原始特征 X (2维)
     |
     ↓
[Hidden Layer] (20 neurons, ReLU)
     |
     ↓
表示 Φ(X) (10维)
```

这个网络将 2 维的原始特征转换成 10 维的 "表示"。希望这个表示能自动捕获非线性关系！

In [ ]:
# 练习 1.2: 简单的表示学习网络

class SimpleRepresentation(nn.Module):
    """
    简单的表示学习网络
    
    X -> [Hidden Layer] -> Phi(X) (学习到的表示)
    """
    
    def __init__(self, input_dim: int, repr_dim: int = 10, hidden_dim: int = 20):
        super().__init__()
        
        # 定义网络层
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, repr_dim),
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """前向传播"""
        return self.network(x)


def train_representation(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    repr_dim: int = 10,
    n_epochs: int = 100
) -> nn.Module:
    """
    训练表示学习网络
    
    目标: 学习能预测 Y 的表示 Phi(X)
    
    训练策略:
    1. Phi(X) -> [Linear] -> Y_pred
    2. 最小化 MSE(Y, Y_pred)
    3. 学到的 Phi(X) 可用于后续因果推断
    """
    
    # 转换为 Tensor
    X_tensor = torch.FloatTensor(X)
    Y_tensor = torch.FloatTensor(Y).unsqueeze(1)
    
    # 模型
    repr_model = SimpleRepresentation(input_dim=X.shape[1], repr_dim=repr_dim)
    
    # 定义预测头 (Phi(X) -> Y)
    prediction_head = nn.Linear(repr_dim, 1)
    
    # 定义优化器和损失函数
    params = list(repr_model.parameters()) + list(prediction_head.parameters())
    optimizer = optim.Adam(params, lr=0.01)
    criterion = nn.MSELoss()
    
    # 训练循环
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        phi = repr_model(X_tensor)
        y_pred = prediction_head(phi)
        loss = criterion(y_pred, Y_tensor)
        loss.backward()
        optimizer.step()
    
    return repr_model

# 测试
repr_model = train_representation(X, T, Y, repr_dim=5, n_epochs=50)
print("表示学习模型创建完成")
print(f"模型结构:")
print(repr_model)

---

## Part 3: 可视化表示空间

### 为什么要可视化?

通过可视化学到的表示，我们可以检查：

1. **表示是否有意义?** 相似的样本应该在表示空间中接近
2. **处理组和控制组是否可区分?** 如果完全分离，可能存在问题
3. **表示是否捕获了重要信息?**

In [ ]:
# 练习 1.3: 可视化表示空间

def visualize_representation(
    repr_model: nn.Module,
    X: np.ndarray,
    T: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """
    可视化学到的表示空间
    
    TODO:
    1. 使用训练好的模型提取表示 Phi(X)
    2. 如果维度 > 2, 使用 PCA 降维到 2D
    3. 返回 2D 表示用于绘图
    
    Returns:
        (phi_2d, T) - 2D 表示和处理标签
    """
    
    repr_model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        
        # TODO: 提取表示
        phi = None  # 你的代码: repr_model(X_tensor)
        
        if phi is None:
            return None, T
        
        phi = phi.numpy()
    
    # TODO: 如果维度 > 2, 使用 PCA 降维
    if phi.shape[1] > 2:
        # 你的代码:
        # pca = PCA(n_components=2)
        # phi = pca.fit_transform(phi)
        pass
    
    return phi, T

# 可视化
if X is not None and repr_model is not None:
    phi_2d, _ = visualize_representation(repr_model, X, T)
    
    if phi_2d is not None:
        plt.figure(figsize=(10, 4))
        
        # 原始特征空间
        plt.subplot(1, 2, 1)
        plt.scatter(X[T==0, 0], X[T==0, 1], alpha=0.5, label='Control', s=10)
        plt.scatter(X[T==1, 0], X[T==1, 1], alpha=0.5, label='Treated', s=10)
        plt.xlabel('X1')
        plt.ylabel('X2')
        plt.title('Original Feature Space')
        plt.legend()
        
        # 表示空间
        plt.subplot(1, 2, 2)
        if phi_2d.shape[1] >= 2:
            plt.scatter(phi_2d[T==0, 0], phi_2d[T==0, 1], alpha=0.5, label='Control', s=10)
            plt.scatter(phi_2d[T==1, 0], phi_2d[T==1, 1], alpha=0.5, label='Treated', s=10)
            plt.xlabel('Phi1')
            plt.ylabel('Phi2')
        plt.title('Learned Representation Space')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("[未完成] 请完成 visualize_representation 函数")

---

## Part 4: 表示平衡性检查

### 什么是表示平衡?

在因果推断中，我们希望处理组和控制组在表示空间中的分布尽可能相似。

如果两组分布差异太大，说明存在 **协变量不平衡**，可能导致估计偏差。

### 平衡性指标

1. **SMD (标准化均值差)**: $\frac{|\bar{\Phi}_T - \bar{\Phi}_C|}{\sigma_{\Phi}}$
   - SMD < 0.1 通常被认为是良好平衡

2. **MMD (最大均值差异)**: $||\bar{\Phi}_T - \bar{\Phi}_C||^2$
   - 衡量两组分布的整体差异

In [ ]:
# 练习 1.4: 表示平衡性检查

def check_representation_balance(
    phi: np.ndarray,
    T: np.ndarray
) -> dict:
    """
    检查表示在处理组和对照组之间的平衡性
    
    平衡性指标:
    1. 标准化均值差 (SMD): |mean(Phi_T) - mean(Phi_C)| / std(Phi)
    2. MMD (Maximum Mean Discrepancy): ||mean(Phi_T) - mean(Phi_C)||^2
    
    TODO: 计算平衡性指标
    
    Returns:
        dict with balance metrics
    """
    
    # 分离处理组和对照组
    phi_treated = phi[T == 1]
    phi_control = phi[T == 0]
    
    # TODO: 计算 SMD (对每个维度)
    # SMD_j = |mean(Phi_T[:, j]) - mean(Phi_C[:, j])| / std(Phi[:, j])
    mean_t = phi_treated.mean(axis=0)
    mean_c = phi_control.mean(axis=0)
    std_all = phi.std(axis=0) + 1e-8  # 避免除零
    
    smd = None  # 你的代码: np.abs(mean_t - mean_c) / std_all
    
    # TODO: 计算 MMD (简化版: 欧氏距离)
    # MMD = ||mean(Phi_T) - mean(Phi_C)||^2
    mmd = None  # 你的代码: np.sum((mean_t - mean_c)**2)
    
    return {
        'smd_mean': np.mean(smd) if smd is not None else None,
        'smd_max': np.max(smd) if smd is not None else None,
        'mmd': mmd
    }

# 测试
if X is not None and repr_model is not None:
    repr_model.eval()
    with torch.no_grad():
        phi = repr_model(torch.FloatTensor(X)).numpy()
    
    balance = check_representation_balance(phi, T)
    if balance['mmd'] is not None:
        print(f"平均 SMD: {balance['smd_mean']:.4f}")
        print(f"最大 SMD: {balance['smd_max']:.4f}")
        print(f"MMD: {balance['mmd']:.4f}")
        print(f"\n提示: SMD < 0.1 表示良好平衡")
    else:
        print("[未完成] 请完成 check_representation_balance 函数")

In [ ]:
# 练习 6.4: 可视化 IPM 与 gamma 参数的关系

def visualize_ipm_sensitivity(
    X: np.ndarray,
    T: np.ndarray
):
    """
    可视化 MMD 对核参数 gamma 的敏感性
    
    TODO: 探索不同 gamma 值对 MMD 的影响
    """
    
    X_treated = X[T == 1]
    X_control = X[T == 0]
    
    # 测试不同的 gamma 值
    gammas = np.logspace(-2, 2, 20)  # 0.01 到 100
    mmds = []
    
    for gamma in gammas:
        mmd = compute_mmd(X_treated, X_control, gamma=gamma)
        mmds.append(mmd)
    
    # 绘图
    plt.figure(figsize=(12, 4))
    
    # 子图 1: MMD vs gamma
    plt.subplot(1, 2, 1)
    plt.plot(gammas, mmds, 'o-', linewidth=2, markersize=4)
    plt.xscale('log')
    plt.xlabel('Gamma (核参数)', fontsize=12)
    plt.ylabel('MMD', fontsize=12)
    plt.title('MMD 对核参数的敏感性', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # 标注最优 gamma
    if len(mmds) > 0 and max(mmds) > 0:
        opt_idx = np.argmax(mmds)
        plt.axvline(gammas[opt_idx], color='red', linestyle='--', alpha=0.5, 
                   label=f'最大 MMD at γ={gammas[opt_idx]:.3f}')
        plt.legend()
    
    # 子图 2: 不同 IPM 对比
    plt.subplot(1, 2, 2)
    
    # 计算不同 IPM
    if repr_model is not None:
        repr_model.eval()
        with torch.no_grad():
            phi = repr_model(torch.FloatTensor(X)).numpy()
        
        phi_t = phi[T == 1]
        phi_c = phi[T == 0]
        
        ipm_values = {
            'MMD\n(γ=0.1)': compute_mmd(phi_t, phi_c, gamma=0.1),
            'MMD\n(γ=0.5)': compute_mmd(phi_t, phi_c, gamma=0.5),
            'MMD\n(γ=2.0)': compute_mmd(phi_t, phi_c, gamma=2.0),
            'Sliced\nWasserstein': sliced_wasserstein_distance(phi_t, phi_c, n_projections=50),
        }
        
        colors = ['#2D9CDB', '#27AE60', '#F2994A', '#EB5757']
        bars = plt.bar(range(len(ipm_values)), list(ipm_values.values()), color=colors, alpha=0.7)
        plt.xticks(range(len(ipm_values)), list(ipm_values.keys()), fontsize=10)
        plt.ylabel('距离值', fontsize=12)
        plt.title('不同 IPM 对比', fontsize=14)
        plt.grid(True, alpha=0.3, axis='y')
        
        # 添加数值标签
        for i, (bar, val) in enumerate(zip(bars, ipm_values.values())):
            plt.text(i, val + 0.001, f'{val:.4f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    print("\n关键发现:")
    print("1. MMD 对 gamma 参数敏感，需要仔细选择")
    print("2. gamma 太小: 核函数太"宽"，区分度差")
    print("3. gamma 太大: 核函数太"窄"，过于敏感")
    print("4. Sliced Wasserstein 不需要调参，更稳健")

# 运行可视化
if X is not None and repr_model is not None:
    visualize_ipm_sensitivity(X, T)

---

## Part 6.4: IPM 与其他方法的联系

### 与 Domain Adaptation（域适应）的关系

**域适应问题**：
- 源域 (Source Domain)：有标签的训练数据
- 目标域 (Target Domain)：无标签的测试数据
- 目标：学习在目标域上表现好的模型

**与因果推断的相似性**：

| 域适应 | 因果推断 |
|--------|---------|
| 源域 | 对照组 |
| 目标域 | 处理组 |
| 标签 | 结果 $Y$ |
| 域迁移 | 处理效应 |

**核心思想都是**：学习一个表示，使得不同"域"（或组）的分布接近！

### Domain Adversarial Neural Network (DANN)

DANN 使用 **对抗训练** 来学习域不变表示：

```
X -> [Shared Encoder Φ] -> Representation
                           /            \
                          /              \
                    [Label Predictor]  [Domain Classifier]
                         (maximize)     (minimize)
```

- **Label Predictor**：预测标签（最大化准确率）
- **Domain Classifier**：预测样本来自哪个域（最小化准确率）

**与因果推断的联系**：
- Domain Classifier 相当于在最小化 IPM
- 当 Domain Classifier 无法区分两域时，IPM 接近 0

### 与 Fair Representation（公平表示）的关系

**公平机器学习问题**：
- 敏感属性 $S$ (如性别、种族)
- 目标：学习不依赖 $S$ 的预测模型

**与因果推断的相似性**：

| 公平表示 | 因果推断 |
|---------|---------|
| 敏感属性 $S$ | 处理 $T$ |
| 不同群体 | 处理组/对照组 |
| 公平性约束 | 平衡性约束 |

**核心思想都是**：学习的表示应该对 $S$（或 $T$）"盲"！

### Demographic Parity via Representation

一种常见的公平性定义是 **Demographic Parity**：

$$P(\hat{Y} = 1 | S = 0) = P(\hat{Y} = 1 | S = 1)$$

**通过表示实现**：

$$P(\Phi(X) | S = 0) = P(\Phi(X) | S = 1)$$

这正是最小化 IPM！

### 统一框架：分布匹配

所有这些方法的核心都是 **分布匹配 (Distribution Matching)**：

$$\min_{\Phi} \underbrace{\mathcal{L}_{\text{task}}(\Phi)}_{\text{任务损失}} + \underbrace{\alpha \cdot \text{IPM}(P_1, P_2)}_{\text{分布匹配}}$$

- **域适应**：$P_1$ = 源域，$P_2$ = 目标域
- **因果推断**：$P_1$ = 处理组，$P_2$ = 对照组
- **公平表示**：$P_1$ = 群体 1，$P_2$ = 群体 2

### 文献连接

1. **TARNet** (Shalit et al., 2017)
   - 第一个在因果推断中使用表示平衡的深度模型
   - 使用 IPM 正则化

2. **DANN** (Ganin et al., 2015)
   - 使用对抗训练进行域适应
   - 理论上等价于最小化 IPM

3. **Fair Representation** (Zemel et al., 2013)
   - 学习对敏感属性不变的表示
   - 使用分布距离作为公平性度量

4. **Wasserstein GAN** (Arjovsky et al., 2017)
   - 使用 Wasserstein 距离训练 GAN
   - 更稳定的对抗训练

In [ ]:
# 练习 6.3: 对比不同 IPM 的效果

def compare_ipm_methods(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    repr_dim: int = 10,
    n_epochs: int = 100
) -> dict:
    """
    对比使用不同 IPM 的表示学习效果
    
    TODO: 实现 IPM 对比实验
    
    比较三种方法:
    1. 无平衡正则化 (alpha = 0)
    2. MMD 平衡正则化
    3. Sliced Wasserstein 平衡正则化
    
    Returns:
        dict with comparison results
    """
    
    from copy import deepcopy
    
    results = {}
    
    # 方法 1: 无平衡正则化
    print("训练无平衡正则化模型...")
    model1 = train_representation(X, T, Y, repr_dim=repr_dim, n_epochs=n_epochs)
    
    model1.eval()
    with torch.no_grad():
        phi1 = model1(torch.FloatTensor(X)).numpy()
    
    phi1_t = phi1[T == 1]
    phi1_c = phi1[T == 0]
    
    results['no_balance'] = {
        'mmd': compute_mmd(phi1_t, phi1_c, gamma=0.5),
        'swd': sliced_wasserstein_distance(phi1_t, phi1_c, n_projections=50),
        'smd_max': np.max(np.abs(phi1_t.mean(0) - phi1_c.mean(0)) / (phi1.std(0) + 1e-8))
    }
    
    # TODO: 方法 2 和 3 留作扩展
    # 在下一个 notebook (TARNet) 中，我们会实现完整的平衡正则化
    
    print("\n当前结果 (无平衡正则化):")
    print(f"MMD: {results['no_balance']['mmd']:.6f}")
    print(f"SWD: {results['no_balance']['swd']:.6f}")
    print(f"Max SMD: {results['no_balance']['smd_max']:.4f}")
    
    return results

# 运行对比实验
if X is not None:
    comparison_results = compare_ipm_methods(X, T, Y, repr_dim=10, n_epochs=50)
    
    print("\n" + "=" * 60)
    print("提示: 在下一个练习 (TARNet) 中，我们会学习如何:")
    print("  1. 在训练中加入 IPM 正则化")
    print("  2. 自动选择最优的 alpha")
    print("  3. 对比不同 IPM 的效果")
    print("=" * 60)

---

## 思考题

### Part 1-5: 基础概念

**1. 为什么线性模型在非线性数据上表现不好?**

**你的答案:**


**2. 表示学习如何帮助因果推断?**

**你的答案:**


**3. 什么是 "表示平衡" (Representation Balance)? 为什么重要?**

**你的答案:**


**4. 在深度因果模型中，共享表示层的作用是什么?**

**你的答案:**


**5. 如果处理组和对照组的表示分布完全不重叠，会有什么问题?**

**你的答案:**


### Part 6: IPM 理论

**6. 什么是 IPM (Integral Probability Metrics)？它的核心思想是什么？**

**你的答案:**


**7. MMD 和 Wasserstein 距离有什么区别？各自适用于什么场景？**

**你的答案:**


**8. 为什么最小化 IPM 能帮助因果推断？请从理论误差界的角度解释。**

**你的答案:**


**9. 在损失函数中，平衡正则化的权重 α 应该如何选择？α 太大或太小会导致什么问题？**

**你的答案:**


**10. 表示学习在因果推断中的应用与域适应、公平表示有什么共同点？**

**你的答案:**


**11. 如果你要设计一个新的因果推断深度模型，你会选择 MMD 还是 Wasserstein 作为平衡损失？为什么？**

**你的答案:**


**12. 在 IPM 误差界中，Lipschitz 常数 λ 代表什么？它如何影响 α 的选择？**

**你的答案:**


In [ ]:
---

## 总结

### 核心概念回顾

| 概念 | 要点 |
|------|------|
| **表示学习** | 让神经网络自动学习有用的特征表示 |
| **非线性关系** | 原始特征的复杂组合可能才是真正有用的 |
| **表示平衡** | 处理组和控制组在表示空间中应该分布相似 |
| **SMD** | 衡量单个维度的不平衡程度 (< 0.1 为良好) |
| **MMD** | 使用核函数度量分布差异，计算效率高 |
| **Wasserstein** | 最优传输距离，考虑几何结构 |
| **IPM** | 分布距离的统一框架，MMD 和 Wasserstein 都是 IPM |
| **误差界** | $\epsilon_{\text{ATE}} \leq \epsilon_{\text{pred}} + \lambda \cdot \text{IPM}$ |
| **正则化权重** | 平衡预测误差和分布匹配，需要调参 |

### IPM 家族对比

| IPM 类型 | 优点 | 缺点 | 适用场景 |
|---------|------|------|---------|
| **MMD (RBF 核)** | 计算快，易实现 | 对核参数敏感 | 中低维数据，快速原型 |
| **Wasserstein** | 几何意义强 | 计算复杂 | 高维数据，图像 |
| **Sliced Wasserstein** | 计算快，近似好 | 需要多次投影 | 高维数据的快速近似 |

### 理论连接

```
        分布匹配 (Distribution Matching)
              /        |         \
             /         |          \
      域适应        因果推断      公平表示
   (Domain Adapt)  (Causal Inf)  (Fair Rep)
        |             |             |
    源域 vs 目标域  处理组 vs 对照组  群体 A vs 群体 B
```

所有这些问题都可以用 **表示学习 + IPM** 来解决！

### 关键公式

**IPM 定义**：
$$\text{IPM}_{\mathcal{F}}(P, Q) = \sup_{f \in \mathcal{F}} \left| \mathbb{E}_{x \sim P}[f(x)] - \mathbb{E}_{x \sim Q}[f(x)] \right|$$

**MMD 核技巧**：
$$\text{MMD}^2(P, Q) = \mathbb{E}_{x, x'}[k(x, x')] - 2\mathbb{E}_{x, y}[k(x, y)] + \mathbb{E}_{y, y'}[k(y, y')]$$

**因果推断损失**：
$$\mathcal{L} = \mathcal{L}_{\text{prediction}} + \alpha \cdot \text{IPM}(P_T, P_C)$$

**误差界**：
$$\epsilon_{\text{ATE}} \leq \epsilon_{\text{prediction}} + \lambda \cdot \text{IPM}(P_T, P_C)$$

### 为什么这很重要?

1. **理论基础**：IPM 为深度因果模型提供了理论保证
2. **实践指导**：指导我们如何设计损失函数和选择超参数
3. **方法统一**：连接了因果推断、域适应、公平表示等多个领域
4. **未来方向**：为 TARNet、DragonNet 等高级模型铺平道路

### 下一步

在下一个练习中，我们将学习 **TARNet** —— 第一个专门为因果推断设计的神经网络！

TARNet 将使用：
- ✅ 共享表示层（本节内容）
- ✅ IPM 平衡正则化（本节内容）
- ➕ 分离的处理效应估计头
- ➕ 端到端的训练策略

---

**恭喜你完成了表示学习与 IPM 理论的深入学习！**

---

## Part 6.2: Wasserstein 距离

### 什么是 Wasserstein 距离？

Wasserstein 距离（也叫 Earth Mover's Distance）有一个直观的物理解释：

**想象两堆沙子**：
- 处理组分布 $P$ 是一堆沙子
- 对照组分布 $Q$ 是另一堆沙子
- Wasserstein 距离是把第一堆沙子 "搬运" 成第二堆的最小工作量

### 数学定义

1-Wasserstein 距离定义为：

$$W_1(P, Q) = \inf_{\gamma \in \Gamma(P, Q)} \mathbb{E}_{(x, y) \sim \gamma}[\|x - y\|]$$

其中 $\Gamma(P, Q)$ 是所有边际分布为 $P$ 和 $Q$ 的联合分布（即所有可能的 "搬运方案"）。

**Kantorovich-Rubinstein 对偶形式**（这是 IPM 形式）：

$$W_1(P, Q) = \sup_{f: \text{Lip}(f) \leq 1} \left| \mathbb{E}_{x \sim P}[f(x)] - \mathbb{E}_{y \sim Q}[f(y)] \right|$$

其中 $\text{Lip}(f) \leq 1$ 表示 $f$ 是 1-Lipschitz 函数（即 $|f(x) - f(y)| \leq \|x - y\|$）。

### Wasserstein vs MMD

| 特性 | MMD | Wasserstein |
|------|-----|-------------|
| **函数类** | RKHS（依赖核函数） | 1-Lipschitz 函数 |
| **几何意义** | 高维空间均值差 | 最优传输距离 |
| **计算复杂度** | $O(n^2)$ | $O(n^3)$ (精确), $O(n^2)$ (近似) |
| **优点** | 易于计算，核函数灵活 | 考虑几何结构，适合图像 |
| **缺点** | 对核参数敏感 | 计算量大 |

### 什么时候用 Wasserstein？

- **高维数据**（如图像）：Wasserstein 更好地捕获几何结构
- **GAN 训练**：Wasserstein GAN (WGAN) 更稳定
- **迁移学习**：域适应问题

### 什么时候用 MMD？

- **中低维数据**：计算效率高
- **因果推断**：表示平衡检验
- **快速原型**：实现简单

In [ ]:
# 练习 6.1: 实现 MMD 损失

def rbf_kernel(X: np.ndarray, Y: np.ndarray, gamma: float = 1.0) -> np.ndarray:
    """
    计算 RBF (高斯) 核矩阵
    
    k(x, y) = exp(-gamma * ||x - y||^2)
    
    TODO: 实现 RBF 核
    
    Args:
        X: (n, d) 矩阵
        Y: (m, d) 矩阵
        gamma: 核参数
    
    Returns:
        K: (n, m) 核矩阵，K[i, j] = k(X[i], Y[j])
    """
    
    # TODO: 计算欧氏距离的平方
    # ||x - y||^2 = ||x||^2 + ||y||^2 - 2<x, y>
    
    XX = None  # 你的代码: np.sum(X**2, axis=1).reshape(-1, 1)  # (n, 1)
    YY = None  # 你的代码: np.sum(Y**2, axis=1).reshape(1, -1)  # (1, m)
    XY = None  # 你的代码: X @ Y.T  # (n, m)
    
    if XX is None:
        return np.zeros((X.shape[0], Y.shape[0]))
    
    dist_sq = XX + YY - 2 * XY
    
    # TODO: 应用高斯核
    K = None  # 你的代码: np.exp(-gamma * dist_sq)
    
    return K if K is not None else np.zeros((X.shape[0], Y.shape[0]))


def compute_mmd(
    X: np.ndarray,
    Y: np.ndarray,
    kernel: str = 'rbf',
    gamma: float = 1.0
) -> float:
    """
    计算 Maximum Mean Discrepancy (MMD)
    
    MMD^2 = E[k(x, x')] - 2*E[k(x, y)] + E[k(y, y')]
    
    TODO: 实现 MMD 计算
    
    Args:
        X: 处理组样本 (n, d)
        Y: 对照组样本 (m, d)
        kernel: 核函数类型
        gamma: 核参数
    
    Returns:
        mmd: MMD 值
    """
    
    n = X.shape[0]
    m = Y.shape[0]
    
    # TODO: 计算三个核矩阵
    K_XX = None  # 你的代码: rbf_kernel(X, X, gamma)
    K_XY = None  # 你的代码: rbf_kernel(X, Y, gamma)
    K_YY = None  # 你的代码: rbf_kernel(Y, Y, gamma)
    
    if K_XX is None:
        return 0.0
    
    # TODO: 计算 MMD^2
    # MMD^2 = (sum(K_XX) - n) / (n*(n-1))  # 去掉对角线
    #       - 2 * sum(K_XY) / (n*m)
    #       + (sum(K_YY) - m) / (m*(m-1))
    
    term1 = None  # 你的代码: (K_XX.sum() - np.trace(K_XX)) / (n * (n - 1))
    term2 = None  # 你的代码: K_XY.sum() / (n * m)
    term3 = None  # 你的代码: (K_YY.sum() - np.trace(K_YY)) / (m * (m - 1))
    
    if term1 is None:
        return 0.0
    
    mmd_sq = term1 - 2 * term2 + term3
    
    # 返回 MMD (开平方)
    return np.sqrt(max(mmd_sq, 0))  # max 确保非负


# 测试 MMD
if X is not None:
    # 提取处理组和对照组的表示
    if repr_model is not None:
        repr_model.eval()
        with torch.no_grad():
            phi = repr_model(torch.FloatTensor(X)).numpy()
        
        phi_treated = phi[T == 1]
        phi_control = phi[T == 0]
        
        # 计算原始特征空间的 MMD
        mmd_original = compute_mmd(X[T == 1], X[T == 0], gamma=0.5)
        
        # 计算表示空间的 MMD
        mmd_repr = compute_mmd(phi_treated, phi_control, gamma=0.5)
        
        if mmd_original > 0:
            print("=" * 50)
            print("MMD 对比分析")
            print("=" * 50)
            print(f"原始特征空间 MMD: {mmd_original:.6f}")
            print(f"表示空间 MMD:     {mmd_repr:.6f}")
            print(f"改进比例: {(1 - mmd_repr/mmd_original)*100:.2f}%")
            print("\n提示: MMD 越小，两组分布越接近")
        else:
            print("[未完成] 请完成 compute_mmd 函数")

---

## Part 6.1: MMD (Maximum Mean Discrepancy) 深入

### 什么是 MMD？

MMD 使用 **核函数** 来度量分布差异。核心思想：

1. 将数据映射到高维特征空间 (通过核函数 $k$)
2. 计算两组在高维空间中的 "平均位置" 的距离
3. 这个距离就是 MMD

### 数学推导

对于两个分布 $P$ (处理组) 和 $Q$ (对照组)：

$$\text{MMD}^2(P, Q) = \left\| \mathbb{E}_{x \sim P}[\phi(x)] - \mathbb{E}_{y \sim Q}[\phi(y)] \right\|^2$$

其中 $\phi$ 是核函数 $k$ 对应的特征映射（通常是无限维的）。

**核技巧 (Kernel Trick)**：我们不需要显式计算 $\phi$，可以直接用核函数：

$$\text{MMD}^2(P, Q) = \mathbb{E}_{x, x'}[k(x, x')] - 2\mathbb{E}_{x, y}[k(x, y)] + \mathbb{E}_{y, y'}[k(y, y')]$$

其中：
- $x, x' \sim P$ (处理组样本)
- $y, y' \sim Q$ (对照组样本)

**通俗理解**：
- 第一项：处理组内部的相似度
- 第二项：处理组和对照组之间的相似度 (乘以 -2)
- 第三项：对照组内部的相似度
- MMD 越小，两组越相似

### 常用核函数

1. **高斯核 (RBF)**：$k(x, y) = \exp(-\gamma \|x - y\|^2)$
   - 最常用，参数 $\gamma$ 控制 "敏感度"
   
2. **线性核**：$k(x, y) = x^T y$
   - 简单，相当于只比较均值

3. **多项式核**：$k(x, y) = (x^T y + c)^d$
   - 可以捕获非线性关系

### IPM 家族成员

不同的函数类 $\mathcal{F}$ 对应不同的 IPM：

| IPM 类型 | 函数类 $\mathcal{F}$ | 特点 | 应用场景 |
|---------|---------------------|------|----------|
| **MMD** | RKHS (再生核希尔伯特空间) | 易于计算，可用核函数 | 表示平衡、分布匹配 |
| **Wasserstein** | 1-Lipschitz 函数 | 考虑"几何距离" | GAN、迁移学习 |
| **Total Variation** | 所有有界函数 | 理论强但难计算 | 理论分析 |

我们重点关注 **MMD** 和 **Wasserstein**，因为它们在深度因果模型中应用最广。

---

## Part 6: IPM 深入理解 - 为什么需要「距离度量」？

### 从直觉到理论

在 Part 4 中，我们使用了 MMD (Maximum Mean Discrepancy) 来衡量处理组和对照组的分布差异。但是：

**为什么要衡量分布差异？**
- 因果推断的核心：处理组和对照组应该 "可比"
- 如果两组分布差异太大，说明存在严重的协变量不平衡
- 表示学习的目标：学习一个表示，使得两组在表示空间中 "接近"

**什么是 IPM (Integral Probability Metrics)？**
- IPM 是一类 **分布距离的度量方法**
- MMD、Wasserstein 距离都属于 IPM 家族
- 它们可以精确量化两个分布的差异

### IPM 的数学定义

对于两个分布 $P$ 和 $Q$，IPM 定义为：

$$\text{IPM}_{\mathcal{F}}(P, Q) = \sup_{f \in \mathcal{F}} \left| \mathbb{E}_{x \sim P}[f(x)] - \mathbb{E}_{x \sim Q}[f(x)] \right|$$

其中 $\mathcal{F}$ 是一个函数类（例如所有有界函数、Lipschitz 函数等）。

**通俗理解**：
- 想象有很多 "探测器" (函数 $f$)，每个探测器给数据打分
- 对于每个探测器，计算两组的平均分数差
- IPM 是所有探测器中，差异最大的那个

---

## Part 5: 对比实验 - 线性 vs 表示学习

让我们对比一下：使用表示学习是否能改善 ATE 估计？

In [ ]:
# 练习 1.5: 对比线性方法和表示学习

def compare_linear_vs_learned(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    true_ate: float = 2.0
) -> dict:
    """
    对比线性方法和表示学习方法
    
    TODO:
    1. 使用线性回归 (原始特征) 估计 ATE
    2. 训练表示学习模型
    3. 使用学到的表示估计 ATE
    4. 比较两种方法的误差
    
    Returns:
        dict with comparison results
    """
    
    # 1. 线性估计
    linear_ate = naive_linear_estimation(X, T, Y)
    
    # 2. 表示学习
    repr_model = train_representation(X, T, Y, repr_dim=10, n_epochs=200)
    
    # 3. 提取表示并估计 ATE
    repr_model.eval()
    with torch.no_grad():
        phi = repr_model(torch.FloatTensor(X)).numpy()
    
    # 用 Phi 替代 X 进行线性回归
    features_phi = np.column_stack([T, phi])
    model = LinearRegression()
    model.fit(features_phi, Y)
    learned_ate = model.coef_[0]  # T 的系数
    
    return {
        'linear_ate': linear_ate,
        'linear_error': abs(linear_ate - true_ate) if linear_ate else None,
        'learned_ate': learned_ate,
        'learned_error': abs(learned_ate - true_ate) if learned_ate else None,
        'true_ate': true_ate
    }

# 运行对比
if X is not None:
    results = compare_linear_vs_learned(X, T, Y, true_ate=2.0)
    if results['linear_ate'] is not None:
        print("=" * 40)
        print("线性方法 vs 表示学习 对比")
        print("=" * 40)
        print(f"真实 ATE: {results['true_ate']:.4f}")
        print(f"\n线性估计: {results['linear_ate']:.4f} (误差: {results['linear_error']:.4f})")
        if results['learned_ate'] is not None:
            print(f"表示学习估计: {results['learned_ate']:.4f} (误差: {results['learned_error']:.4f})")
    else:
        print("[未完成] 请完成所有前序练习")

---

## 思考题

### 1. 为什么线性模型在非线性数据上表现不好?

**你的答案:**


### 2. 表示学习如何帮助因果推断?

**你的答案:**


### 3. 什么是 "表示平衡" (Representation Balance)? 为什么重要?

**你的答案:**


### 4. 在深度因果模型中，共享表示层的作用是什么?

**你的答案:**


### 5. 如果处理组和对照组的表示分布完全不重叠，会有什么问题?

**你的答案:**


---

## 总结

| 概念 | 要点 |
|------|------|
| **表示学习** | 让神经网络自动学习有用的特征表示 |
| **非线性关系** | 原始特征的复杂组合可能才是真正有用的 |
| **表示平衡** | 处理组和控制组在表示空间中应该分布相似 |
| **SMD** | 衡量单个维度的不平衡程度 |
| **MMD** | 衡量整体分布差异 |

### 为什么这很重要?

表示学习是深度因果模型的基础。在下一个练习中，我们将学习 **TARNet** —— 第一个专门为因果推断设计的神经网络！

---

*恭喜你完成了表示学习基础的学习！* 🎉